In [1]:
import pandas as pd
from z3 import *
from time import time
from tqdm import tqdm

floors = set(range(4, 22))

male_floors = set([8, 10, 16, 19])
female_floors = set([7, 14, 18, 20])
mixed_floors = floors.difference(male_floors).difference(female_floors)

rf_floors = set(range(5, 22, 4))
laundry_floors = set([9, 17])

freshmen_floors = set([6, 8, 11, 14, 16, 17, 20])
senior_floors = floors.difference(freshmen_floors)

corridor_nonaircon_pos = set(map(str, range(101, 107) + range(115, 120)))
corridor_aircon_pos = set(map(str, range(109, 115)))
suite_room_letters = map(lambda x: chr(ord('A') + x), range(6))
suite_nonaircon_pos = set([str(suite_num) + letter for suite_num in [100, 107] for letter in suite_room_letters])
suite_aircon_pos = set([str(suite_num) + letter for suite_num in [108] for letter in suite_room_letters])
aircon_pos = corridor_aircon_pos.union(suite_aircon_pos)
nonaircon_pos = corridor_nonaircon_pos.union(suite_nonaircon_pos)
corridor_pos = corridor_nonaircon_pos.union(corridor_aircon_pos)
suite_room_pos = suite_nonaircon_pos.union(suite_aircon_pos)
absent_laundry_pos = set(map(str, range(103, 107)))
room_pos = corridor_pos.union(suite_room_pos)

def get_valid_floors(is_male, pref_mixed, is_freshmen):
    gender_floors = male_floors if is_male else female_floors
    gender_floors = gender_floors.union(mixed_floors) if pref_mixed else gender_floors
    seniority_floors = freshmen_floors if is_freshmen else senior_floors
    return gender_floors.intersection(seniority_floors)

def get_valid_pos(floor_num, is_suite, is_aircon):
    pos = room_pos
    pos = pos.difference(corridor_pos) if is_suite else pos.difference(suite_room_pos)
    pos = pos.difference(nonaircon_pos) if is_aircon else pos.difference(aircon_pos)
        
    if floor_num not in floors:
        pos = pos.difference(room_pos)
        
    if floor_num in rf_floors:
        pos = pos.difference(suite_aircon_pos)
    
    if floor_num in laundry_floors:
        pos = pos.difference(absent_laundry_pos)

    return pos

def get_valid_rooms(is_male=True, pref_mixed=False, is_freshmen=False, is_suite=True, is_aircon=False):
    valid_floors = get_valid_floors(is_male, pref_mixed, is_freshmen)
    valid_rooms = [str(floor_num) + '-' + pos for floor_num in valid_floors for pos in get_valid_pos(floor_num, is_suite, is_aircon)]    
    return set(valid_rooms)

def get_all_rooms():
    rooms = []
    for floor_num in floors:
        if floor_num in laundry_floors:
            rooms.extend([str(floor_num) + '-' + pos for pos in room_pos.difference(absent_laundry_pos)])
        elif floor_num in rf_floors:
            rooms.extend([str(floor_num) + '-' + pos for pos in room_pos.difference(suite_aircon_pos)])
        else:
            rooms.extend([str(floor_num) + '-' + pos for pos in room_pos])
    return rooms
    
class Person():
    def __init__(self, name, gender, pref_mixed, usp_status, is_suite, is_aircon, floor_pref, room_pref):
        self.name = name
        self.gender = gender
        self.pref_mixed = pref_mixed
        self.usp_status = usp_status
        self.is_suite = is_suite
        self.is_aircon = is_aircon
        self.floor_pref = floor_pref
        self.room_pref = room_pref
    
    def __str__(self):
        return self.name
    
    def valid_pos(self):
        return get_valid_pos(16,
                             self.is_suite == "suite",
                             self.is_aircon == "aircon"
                            )
        
    def valid_floors(self):
        return get_valid_floors(self.gender == "male",
                               self.pref_mixed == "yes",
                               self.usp_status == "freshmen"
                               )
    
    def valid_rooms(self):
        return get_valid_rooms(self.gender == "male",
                               self.pref_mixed == "yes",
                               self.usp_status == "freshmen",
                               self.is_suite == "suite",
                               self.is_aircon == "aircon"
                              )
    
    __repr__ = __str__

In [2]:
def get_people_constraints(people):
    rooms = set()
    opc = []
    pc = []
    basic_pc = []
    for person in tqdm(people):
        if person.floor_pref and person.room_pref:
            sym = Int(person.name + "-" + person.floor_pref + '-' + person.room_pref) == 1
            opc.append(sym)
        elif person.floor_pref:
            syms = Sum([Int(person.name + "-" + person.floor_pref + '-' + room) for room in person.valid_pos()]) == 1
            opc.append(syms)
        elif person.room_pref:
            syms = []
            for floor_pref in person.valid_floors():
                if person.room_pref in suite_aircon_pos and floor_pref in rf_floors:
                    continue
                
                if person.room_pref in absent_laundry_pos and floor_pref in laundry_floors:
                    continue
                
                sym = Int(person.name + "-" + str(floor_pref) + '-' + person.room_pref)
                syms.append(sym)

            opc.append(Sum(syms) == 1)
        
        p_valid_rooms = person.valid_rooms()
        rooms = rooms.union(p_valid_rooms)

        rs = [Int(person.name + "-" + room) for room in p_valid_rooms]
        pc.append(Sum(rs) == 1)
        basic_pc.extend([Int(person.name + "-" + room) >= 0 for room in person.valid_rooms()])
#         basic_pc.extend([Int(person.name + "-" + room) <= 1 for room in person.valid_rooms()])
#         basic_pc.extend([Int(person.name + "-" + room) > 0 for room in person.valid_rooms()])

    return basic_pc, pc, opc, rooms

def get_room_constraints(people, rooms):
    basic_rc = []
    rc = []                            
    for room in tqdm(rooms):
        basic_rc.extend([Int(person.name + "-" + room) >= 0 for person in people])
        basic_rc.extend([Int(person.name + "-" + room) <= 1 for person in people])        
        ps = [Int(person.name + "-" + room) for person in people]
        rc.append(Sum(ps) >= 0)
        rc.append(Sum(ps) <= 1)
    
    return basic_rc, rc
    
def write_assignments(input_file, assignments):
    d = dict()
    for a in assignments:
        name, floor, room = obj_to_string(a).split('-')
        d[name] = [floor, room]
    
    input_file_values = input_file.iloc[:].values
    output_file_array = []
    for row in input_file_values:
        row_name = str(row[0])
        row_floor, row_room = d[row_name]
        out_row = list(row)
        out_row[-2] = row_floor
        out_row[-1] = row_room
        output_file_array.append(out_row)

    out_df = pd.DataFrame(output_file_array)
    out_df.columns = input_file.columns
    out_df.to_csv('output.csv', index=False)
    
    return out_df

def view_times(times):
    if len(times) < 1:
        return
    last_t = times[0][1]
    new_times = []
    for t in times:
        new_times.append([t[0], t[1] - last_t])
        last_t = t[1]

    return pd.DataFrame(new_times)

In [3]:
g = 1
def solve(nrows=None):
    times = []
    times.append(["start time", time()])
    input_file = pd.read_csv('simple.csv', nrows=nrows, dtype=object).fillna('')
    times.append(["read file", time()])
    print times[-1][0], times[-1][1] - times[-2][1]

    people = []
    for row in input_file.iloc[:].values:
        person = Person(*row)
        people.append(person)

    prs = [Int(person.name + "-" + room) for person in people for room in person.valid_rooms()]
    basic_pc, pc, opc, rooms = get_people_constraints(people)
    times.append(["get people constraints", time()])
    print times[-1][0], times[-1][1] - times[-2][1]
    
    basic_rc, rc = get_room_constraints(people, rooms)
    times.append(["get room constraints", time()])
    print times[-1][0], times[-1][1] - times[-2][1]

    opt = Optimize()
    opt.add(basic_pc)
    opt.add(pc)
    opt.add(basic_rc)
    opt.add(rc)
    for op in opc:
        opt.add_soft(op)
    times.append(["add constraints to optimizer", time()])
    print times[-1][0], times[-1][1] - times[-2][1]

    sat_result = opt.check()
    times.append(["do sat check", time()])
    print times[-1][0], times[-1][1] - times[-2][1]
    
    if sat_result == sat:
        global g
        model = opt.model()
        g = model
        assignments = filter(lambda x: model.eval(x).as_long() > 0, prs)
        w = write_assignments(input_file, assignments)
        times.append(["write assignments", time()])
        print times[-1][0], times[-1][1] - times[-2][1]
        return w, times
    else:
        return "No solution found", times

In [ ]:
out_df, times = solve(50)
view_times(times)


read file 0.00768613815308
get people constraints 0.874548912048
get room constraints 13.7784559727
add constraints to optimizer 0.253806114197


In [5]:
out_df

,name,gender,pref_mixed,usp_status,suite/corridor,aircon/nonaircon,floor_pref,room_pref
0,1,female,yes,senior,corridor,no,7,106
1,2,male,yes,freshmen,corridor,no,8,118
2,3,female,yes,senior,corridor,no,5,102
3,4,male,no,senior,suite,aircon,10,108F
4,5,female,yes,senior,suite,aircon,15,108C
5,6,female,yes,senior,corridor,no,13,116
6,7,male,yes,freshmen,corridor,no,6,105
7,8,female,yes,senior,corridor,no,13,103
8,9,male,no,senior,suite,aircon,19,108F
9,10,female,yes,senior,suite,aircon,18,108A


In [63]:
g.eval(Int('44-10-108E'))

1

In [55]:
x = get_all_rooms()

In [64]:
x.index('10-108E')

229

In [69]:
input_file = pd.read_csv('simple.csv', nrows=50, dtype=object).fillna('')
people = []
for row in input_file.iloc[:].values:
    person = Person(*row)
    people.append(person)

prs = [Int(person.name + "-" + room) for person in people for room in person.valid_rooms()]
basic_pc, pc, opc = get_people_constraints(people)

basic_rc, rc = get_room_constraints(people)

write assignments 0.206997156143


In [76]:
rc[229*2+1]

1-10-108E +
2-10-108E +
3-10-108E +
4-10-108E +
5-10-108E +
6-10-108E +
7-10-108E +
8-10-108E +
9-10-108E +
10-10-108E +
11-10-108E +
12-10-108E +
13-10-108E +
14-10-108E +
15-10-108E +
16-10-108E +
17-10-108E +
18-10-108E +
19-10-108E +
20-10-108E +
21-10-108E +
22-10-108E +
23-10-108E +
24-10-108E +
25-10-108E +
26-10-108E +
27-10-108E +
28-10-108E +
29-10-108E +
30-10-108E +
31-10-108E +
32-10-108E +
33-10-108E +
34-10-108E +
35-10-108E +
36-10-108E +
37-10-108E +
38-10-108E +
39-10-108E +
40-10-108E +
41-10-108E +
42-10-108E +
43-10-108E +
44-10-108E +
45-10-108E +
46-10-108E +
47-10-108E +
48-10-108E +
49-10-108E +
50-10-108E <=
1